In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
import re

In [3]:
# Load the data
df = pd.read_csv('DataNeuron_text_similarity.csv')

In [4]:
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [5]:
# Text cleaning function using re
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and digits using regex
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [6]:
# Apply text cleaning to 'text1' and 'text2'
df['text1'] = df['text1'].apply(clean_text)
df['text2'] = df['text2'].apply(clean_text)

In [7]:
# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [8]:
# Tokenize input text and get DistilBERT embeddings
def get_distilbert_embeddings(text):
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
    outputs = distilbert_model(inputs)
    return outputs.last_hidden_state[:, 0, :]

In [9]:
# Tokenize and get DistilBERT embeddings for text1 and text2
X_text1 = np.vstack(df['text1'].apply(get_distilbert_embeddings).values)
X_text2 = np.vstack(df['text2'].apply(get_distilbert_embeddings).values)

In [10]:
# Model Architecture
input_text1 = Input(shape=(768,))
input_text2 = Input(shape=(768,))
merged_layer = Concatenate()([input_text1, input_text2])
dense1 = Dense(128, activation='relu')(merged_layer)
dense2 = Dense(64, activation='relu')(dense1)
output_layer = Dense(1, activation='sigmoid')(dense2)

model = Model(inputs=[input_text1, input_text2], outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])


In [26]:
# Train the model
history = model.fit([X_text1, X_text2], np.zeros(len(df)), epochs=3000, batch_size=32)  


Epoch 1/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7613e-14 - mse: 6.7613e-14
Epoch 2/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7610e-14 - mse: 6.7610e-14
Epoch 3/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7607e-14 - mse: 6.7607e-14
Epoch 4/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7604e-14 - mse: 6.7604e-14
Epoch 5/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7601e-14 - mse: 6.7601e-14
Epoch 6/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7598e-14 - mse: 6.7598e-14
Epoch 7/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7595e-14 - mse: 6.7595e-14
Epoch 8/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7592e-14 - mse: 6.7592e-14
Epoch 9/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7589e-14 - mse: 6.7589e-14
Epoch 10/3000
94/94 [==============================] - 0s 1ms/step - loss

94/94 [==============================] - 0s 1ms/step - loss: 6.7148e-14 - mse: 6.7148e-14
Epoch 158/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7145e-14 - mse: 6.7145e-14
Epoch 159/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7142e-14 - mse: 6.7142e-14
Epoch 160/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7139e-14 - mse: 6.7139e-14
Epoch 161/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7136e-14 - mse: 6.7136e-14
Epoch 162/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7133e-14 - mse: 6.7133e-14
Epoch 163/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7130e-14 - mse: 6.7130e-14
Epoch 164/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7127e-14 - mse: 6.7127e-14
Epoch 165/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.7124e-14 - mse: 6.7124e-14
Epoch 166/3000
94/94 [==============================] - 0s 1ms/step - 

94/94 [==============================] - 0s 997us/step - loss: 6.6691e-14 - mse: 6.6691e-14
Epoch 314/3000
94/94 [==============================] - 0s 999us/step - loss: 6.6688e-14 - mse: 6.6688e-14
Epoch 315/3000
94/94 [==============================] - 0s 997us/step - loss: 6.6685e-14 - mse: 6.6685e-14
Epoch 316/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6682e-14 - mse: 6.6682e-14
Epoch 317/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6679e-14 - mse: 6.6679e-14
Epoch 318/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6677e-14 - mse: 6.6677e-14
Epoch 319/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6674e-14 - mse: 6.6674e-14
Epoch 320/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6671e-14 - mse: 6.6671e-14
Epoch 321/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6668e-14 - mse: 6.6668e-14
Epoch 322/3000
94/94 [==============================] - 0s 1ms/s

94/94 [==============================] - 0s 1ms/step - loss: 6.6465e-14 - mse: 6.6465e-14
Epoch 392/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6462e-14 - mse: 6.6462e-14
Epoch 393/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6459e-14 - mse: 6.6459e-14
Epoch 394/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6456e-14 - mse: 6.6456e-14
Epoch 395/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6453e-14 - mse: 6.6453e-14
Epoch 396/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6451e-14 - mse: 6.6451e-14
Epoch 397/3000
94/94 [==============================] - 0s 2ms/step - loss: 6.6448e-14 - mse: 6.6448e-14
Epoch 398/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6445e-14 - mse: 6.6445e-14
Epoch 399/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6442e-14 - mse: 6.6442e-14
Epoch 400/3000
94/94 [==============================] - 0s 1ms/step - 

94/94 [==============================] - 0s 1ms/step - loss: 6.6020e-14 - mse: 6.6020e-14
Epoch 548/3000
94/94 [==============================] - 0s 2ms/step - loss: 6.6017e-14 - mse: 6.6017e-14
Epoch 549/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6014e-14 - mse: 6.6014e-14
Epoch 550/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6011e-14 - mse: 6.6011e-14
Epoch 551/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6009e-14 - mse: 6.6009e-14
Epoch 552/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6006e-14 - mse: 6.6006e-14
Epoch 553/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6003e-14 - mse: 6.6003e-14
Epoch 554/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.6000e-14 - mse: 6.6000e-14
Epoch 555/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.5997e-14 - mse: 6.5997e-14
Epoch 556/3000
94/94 [==============================] - 0s 1ms/step - 

...

94/94 [==============================] - 0s 1ms/step - loss: 6.1655e-14 - mse: 6.1655e-14
Epoch 2248/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1652e-14 - mse: 6.1652e-14
Epoch 2249/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1650e-14 - mse: 6.1650e-14
Epoch 2250/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1648e-14 - mse: 6.1648e-14
Epoch 2251/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1646e-14 - mse: 6.1646e-14
Epoch 2252/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1643e-14 - mse: 6.1643e-14
Epoch 2253/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1641e-14 - mse: 6.1641e-14
Epoch 2254/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1639e-14 - mse: 6.1639e-14
Epoch 2255/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1636e-14 - mse: 6.1636e-14
Epoch 2256/3000
94/94 [==============================] - 0s 1m

94/94 [==============================] - 0s 1ms/step - loss: 6.1301e-14 - mse: 6.1301e-14
Epoch 2402/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1298e-14 - mse: 6.1298e-14
Epoch 2403/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1296e-14 - mse: 6.1296e-14
Epoch 2404/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1294e-14 - mse: 6.1294e-14
Epoch 2405/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1292e-14 - mse: 6.1292e-14
Epoch 2406/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1289e-14 - mse: 6.1289e-14
Epoch 2407/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1287e-14 - mse: 6.1287e-14
Epoch 2408/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1285e-14 - mse: 6.1285e-14
Epoch 2409/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.1283e-14 - mse: 6.1283e-14
Epoch 2410/3000
94/94 [==============================] - 0s 1m

94/94 [==============================] - 0s 1ms/step - loss: 6.0954e-14 - mse: 6.0954e-14
Epoch 2556/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0952e-14 - mse: 6.0952e-14
Epoch 2557/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0949e-14 - mse: 6.0949e-14
Epoch 2558/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0947e-14 - mse: 6.0947e-14
Epoch 2559/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0945e-14 - mse: 6.0945e-14
Epoch 2560/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0943e-14 - mse: 6.0943e-14
Epoch 2561/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0940e-14 - mse: 6.0940e-14
Epoch 2562/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0938e-14 - mse: 6.0938e-14
Epoch 2563/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0936e-14 - mse: 6.0936e-14
Epoch 2564/3000
94/94 [==============================] - 0s 1m

94/94 [==============================] - 0s 1ms/step - loss: 6.0613e-14 - mse: 6.0613e-14
Epoch 2710/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0611e-14 - mse: 6.0611e-14
Epoch 2711/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0609e-14 - mse: 6.0609e-14
Epoch 2712/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0607e-14 - mse: 6.0607e-14
Epoch 2713/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0605e-14 - mse: 6.0605e-14
Epoch 2714/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0602e-14 - mse: 6.0602e-14
Epoch 2715/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0600e-14 - mse: 6.0600e-14
Epoch 2716/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0598e-14 - mse: 6.0598e-14
Epoch 2717/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0596e-14 - mse: 6.0596e-14
Epoch 2718/3000
94/94 [==============================] - 0s 1m

94/94 [==============================] - 0s 1ms/step - loss: 6.0279e-14 - mse: 6.0279e-14
Epoch 2864/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0277e-14 - mse: 6.0277e-14
Epoch 2865/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0274e-14 - mse: 6.0274e-14
Epoch 2866/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0272e-14 - mse: 6.0272e-14
Epoch 2867/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0270e-14 - mse: 6.0270e-14
Epoch 2868/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0268e-14 - mse: 6.0268e-14
Epoch 2869/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0266e-14 - mse: 6.0266e-14
Epoch 2870/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0264e-14 - mse: 6.0264e-14
Epoch 2871/3000
94/94 [==============================] - 0s 1ms/step - loss: 6.0262e-14 - mse: 6.0262e-14
Epoch 2872/3000
94/94 [==============================] - 0s 1m

In [27]:
# Predict similarity scores
predictions = model.predict([X_text1, X_text2])


94/94 [==============================] - 0s 695us/step


In [28]:
# Add the predicted similarity scores to the DataFrame
df['similarity'] = predictions

In [31]:
df.head()

,text1,text2,similarity
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,1.378889e-07
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,1.818138e-07
2,player burnout worries robinson england coach ...,hanks greeted at wintry premiere hollywood sta...,1.573878e-07
3,hearts of oak cotonsport hearts of oak set up...,redford s vision of sundance despite sporting ...,1.517882e-07
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,3.944085e-07


In [30]:
# Save the trained model
model.save('distilbert_similarity_model')

INFO:tensorflow:Assets written to: distilbert_similarity_model/assets


INFO:tensorflow:Assets written to: distilbert_similarity_model/assets
